In [1]:
import os
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
import libfmp.c3
import librosa
from numba import jit, njit
import scipy.spatial.distance as dist
from scipy.io import wavfile
import IPython.display as ipyd
import scipy.spatial.distance as dist
from pydub import AudioSegment
import time

# Ideia inicial:
Descobrir o quão util pode ser a DTW para reconhecimento de palavras/frases:
- Custo de alinhamento normalizado x não normalizado.
- comparar qual espectograma funciona melhor mel x chroma

# Em Chroma:

In [3]:
N, H = 4096, 512
#selecionando o primeiro audio:
x, f_s_x = librosa.load("clips/common_voice_pt_19273358.mp3")
# Chroma spectrogram do primeiro:
X = librosa.feature.chroma_stft(y=x, sr=f_s_x, tuning=0, norm=2, hop_length=H, n_fft=N)
ipyd.Audio(rate=f_s_x, data=x)
x_seq = X.T
#Comparando com a base de dados:
audio_dir = "clips/"
selected_audios = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".mp3")][:5000]
smallest_costs = [float('inf')] * 10
smallest_audios = [None] * 10
start_time = time.time()
for audio in selected_audios:
    y, f_s_y = librosa.load(audio)
    Y = librosa.feature.chroma_stft(y=y, sr=f_s_y, tuning=0, norm=2, hop_length=H, n_fft=N)
    y_seq = Y.T
    C = libfmp.c3.compute_cost_matrix(X, Y)
    D = libfmp.c3.compute_accumulated_cost_matrix(C)
    P = libfmp.c3.compute_optimal_warping_path(D)
    alignment_cost = D[-1, -1]
    normalized_cost = alignment_cost / (y_seq.shape[0] + x_seq.shape[0])
    
    # Check if the current audio has a smaller cost than any of the current smallest costs
    for i in range(10):
        if normalized_cost < smallest_costs[i]:
            smallest_costs.insert(i, normalized_cost)
            smallest_costs = smallest_costs[:10]
            smallest_audios.insert(i, audio)
            smallest_audios = smallest_audios[:10]
            break
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução do loop: {execution_time} segundos")
# Print the 10 audios with the smallest alignment cost
for i, audio in enumerate(smallest_audios):
    print(f"Audio {i+1} - File: {audio}, Normalized Alignment Cost: {smallest_costs[i]}")

Tempo de execução do loop: 96.67957639694214 segundos
Audio 1 - File: clips/common_voice_pt_19273358.mp3, Normalized Alignment Cost: 0.0
Audio 2 - File: clips/common_voice_pt_19338125.mp3, Normalized Alignment Cost: 0.2685436809366923
Audio 3 - File: clips/common_voice_pt_19372960.mp3, Normalized Alignment Cost: 0.27197793547104065
Audio 4 - File: clips/common_voice_pt_19369652.mp3, Normalized Alignment Cost: 0.2735519771141448
Audio 5 - File: clips/common_voice_pt_19369678.mp3, Normalized Alignment Cost: 0.27901505109119357
Audio 6 - File: clips/common_voice_pt_19331134.mp3, Normalized Alignment Cost: 0.27918380578051427
Audio 7 - File: clips/common_voice_pt_19372678.mp3, Normalized Alignment Cost: 0.2801055062462347
Audio 8 - File: clips/common_voice_pt_19331105.mp3, Normalized Alignment Cost: 0.28319949372282793
Audio 9 - File: clips/common_voice_pt_19347170.mp3, Normalized Alignment Cost: 0.28520041141141866
Audio 10 - File: clips/common_voice_pt_19377035.mp3, Normalized Alignment 

Não há semelhança nenhuma entre os audios.

# matriz de custo gerada através da formula do cosseno:

In [5]:
N, H = 4096, 512
#selecionando o primeiro audio:
x, f_s_x = librosa.load("clips/common_voice_pt_19273358.mp3")
# Chroma spectrogram do primeiro:
X = librosa.feature.chroma_stft(y=x, sr=f_s_x, tuning=0, norm=2, hop_length=H, n_fft=N)
ipyd.Audio(rate=f_s_x, data=x)
x_seq = X.T
#Comparando com a base de dados:
audio_dir = "clips/"
selected_audios = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".mp3")][:5000]
smallest_costs = [float('inf')] * 10
smallest_audios = [None] * 10
start_time = time.time()
for audio in selected_audios:
    y, f_s_y = librosa.load(audio)
    Y = librosa.feature.chroma_stft(y=y, sr=f_s_y, tuning=0, norm=2, hop_length=H, n_fft=N)
    y_seq = Y.T
    C = dist.cdist(x_seq, y_seq, "cosine")
    #C = libfmp.c3.compute_cost_matrix(X, Y)
    D = libfmp.c3.compute_accumulated_cost_matrix(C)
    P = libfmp.c3.compute_optimal_warping_path(D)
    alignment_cost = D[-1, -1]
    normalized_cost = alignment_cost / (y_seq.shape[0] + x_seq.shape[0])
    
    # Check if the current audio has a smaller cost than any of the current smallest costs
    for i in range(10):
        if normalized_cost < smallest_costs[i]:
            smallest_costs.insert(i, normalized_cost)
            smallest_costs = smallest_costs[:10]
            smallest_audios.insert(i, audio)
            smallest_audios = smallest_audios[:10]
            break
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução do loop: {execution_time} segundos")
# Print the 10 audios with the smallest alignment cost
for i, audio in enumerate(smallest_audios):
    print(f"Audio {i+1} - File: {audio}, Normalized Alignment Cost: {smallest_costs[i]}")

Tempo de execução do loop: 101.02801489830017 segundos
Audio 1 - File: clips/common_voice_pt_19273358.mp3, Normalized Alignment Cost: 1.4426626873660226e-17
Audio 2 - File: clips/common_voice_pt_19338125.mp3, Normalized Alignment Cost: 0.06863948390711373
Audio 3 - File: clips/common_voice_pt_19369652.mp3, Normalized Alignment Cost: 0.07003669584041818
Audio 4 - File: clips/common_voice_pt_19331134.mp3, Normalized Alignment Cost: 0.07072787357176913
Audio 5 - File: clips/common_voice_pt_19347170.mp3, Normalized Alignment Cost: 0.07102583596871946
Audio 6 - File: clips/common_voice_pt_19331123.mp3, Normalized Alignment Cost: 0.07477681167441243
Audio 7 - File: clips/common_voice_pt_19372960.mp3, Normalized Alignment Cost: 0.07512247545764633
Audio 8 - File: clips/common_voice_pt_19372678.mp3, Normalized Alignment Cost: 0.07655678075878566
Audio 9 - File: clips/common_voice_pt_19401216.mp3, Normalized Alignment Cost: 0.07680902861296908
Audio 10 - File: clips/common_voice_pt_19401533.mp3

Os audios com o custo de alinhamento menor, são os que possuem muito tempo em silencio.

# Melspectogram:

In [9]:
N, H = 4096, 512
#selecionando o primeiro audio:
x, f_s_x = librosa.load("clips/common_voice_pt_19273358.mp3")
# Chroma spectrogram do primeiro:
X = librosa.feature.melspectrogram(y=x, sr=f_s_x, hop_length=H, n_fft=N)
ipyd.Audio(rate=f_s_x, data=x)
x_seq = X.T
#Comparando com a base de dados:
audio_dir = "clips/"
selected_audios = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".mp3")][:5000]
smallest_costs = [float('inf')] * 10
smallest_audios = [None] * 10
start_time = time.time()
for audio in selected_audios:
    y, f_s_y = librosa.load(audio)
    Y = librosa.feature.melspectrogram(y=y, sr=f_s_y, hop_length=H, n_fft=N)
    y_seq = Y.T
    C = dist.cdist(x_seq, y_seq, "cosine")
    #C = libfmp.c3.compute_cost_matrix(X, Y)
    D = libfmp.c3.compute_accumulated_cost_matrix(C)
    P = libfmp.c3.compute_optimal_warping_path(D)
    alignment_cost = D[-1, -1]
    normalized_cost = alignment_cost / (y_seq.shape[0] + x_seq.shape[0])
    
    # Check if the current audio has a smaller cost than any of the current smallest costs
    for i in range(10):
        if normalized_cost < smallest_costs[i]:
            smallest_costs.insert(i, normalized_cost)
            smallest_costs = smallest_costs[:10]
            smallest_audios.insert(i, audio)
            smallest_audios = smallest_audios[:10]
            break
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução do loop: {execution_time} segundos")
# Print the 10 audios with the smallest alignment cost
for i, audio in enumerate(smallest_audios):
    print(f"Audio {i+1} - File: {audio}, Normalized Alignment Cost: {smallest_costs[i]}")

Tempo de execução do loop: 131.20780324935913 segundos
Audio 1 - File: clips/common_voice_pt_19273358.mp3, Normalized Alignment Cost: 1.2544892933617588e-17
Audio 2 - File: clips/common_voice_pt_19273362.mp3, Normalized Alignment Cost: 0.2421714480237182
Audio 3 - File: clips/common_voice_pt_19338507.mp3, Normalized Alignment Cost: 0.24523767083111386
Audio 4 - File: clips/common_voice_pt_19379766.mp3, Normalized Alignment Cost: 0.24620468237678728
Audio 5 - File: clips/common_voice_pt_19335437.mp3, Normalized Alignment Cost: 0.24998157059386403
Audio 6 - File: clips/common_voice_pt_19331454.mp3, Normalized Alignment Cost: 0.2539998449683059
Audio 7 - File: clips/common_voice_pt_19338487.mp3, Normalized Alignment Cost: 0.2553265189914809
Audio 8 - File: clips/common_voice_pt_19338601.mp3, Normalized Alignment Cost: 0.2573090209291416
Audio 9 - File: clips/common_voice_pt_19372578.mp3, Normalized Alignment Cost: 0.25795736433407257
Audio 10 - File: clips/common_voice_pt_19335446.mp3, No

Sem nenhuma mudança expressiva, apenas que no primeiro audio temos a certeza de que é a mesma pessoa falando. Possivelmente a melhor escolha para achar os audios de uma pessoa num database.